In [113]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [114]:
from tensorflow.keras.preprocessing.text import one_hot

In [115]:
import pandas as pd
import numpy as np
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [116]:
df = pd.read_csv(r'C:\Users\satya\Downloads\train_E6oV3lV.csv')
df

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."


In [117]:
def preprocess2(tweet, new_col):
    def remove_pattern(input_txt, pattern):
        r = re.findall(pattern, input_txt)
        for word in r:
            input_txt = re.sub(word, "", input_txt)
        return input_txt
    df[new_col] = np.vectorize(remove_pattern)(tweet, "@[\w]*")
    df[new_col] = df[new_col].str.replace("[^a-zA-Z#]", " ")
    df[new_col] = df[new_col].apply(lambda x: " ".join([w for w in x.split() if len(w)>3]))
    tokenized_tweet = df[new_col].apply(lambda x: x.split())
    from nltk.stem.porter import PorterStemmer
    stemmer = PorterStemmer()

    tokenized_tweet = tokenized_tweet.apply(lambda sentence: [stemmer.stem(word) for word in sentence])
    for i in range(len(tokenized_tweet)):
        tokenized_tweet[i] = " ".join(tokenized_tweet[i])
    
    df[new_col] = tokenized_tweet
    return df[new_col]

In [118]:
 preprocess2(df['tweet'],'clean_tweet2')

C:\Users\satya\AppData\Local\Temp\ipykernel_18084\2353425536.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df[new_col] = df[new_col].str.replace("[^a-zA-Z#]", " ")


0        when father dysfunct selfish drag kid into dys...
1        thank #lyft credit caus they offer wheelchair ...
2                                      bihday your majesti
3                               #model love take with time
4                                 factsguid societi #motiv
                               ...                        
31957                                           that youuu
31958    nina turner airwav tri wrap herself mantl genu...
31959                         listen song monday morn work
31960          #sikh #templ vandalis #calgari #wso condemn
31961                                         thank follow
Name: clean_tweet2, Length: 31962, dtype: object

In [119]:
voc_size=100000

In [120]:
onehot_repr=[one_hot(tweet,voc_size)for tweet in df['tweet']] 
print(onehot_repr)

[[40405, 25170, 99182, 40353, 21951, 56856, 27042, 21951, 96559, 51744, 2128, 63928, 7220, 78975, 89698, 7220, 80131, 53232], [40405, 40405, 50002, 83680, 43965, 63933, 38587, 94071, 11402, 41229, 62702, 52646, 41894, 91113, 39469, 74041, 14652, 57956, 82532], [37556, 80621, 67617], [23264, 38587, 48903, 12027, 3025, 63570, 12027, 29859, 98230, 38283, 74041, 14938, 60439], [83910, 76109, 7861, 34197], [94179, 94179, 79205, 61197, 21376, 27042, 75938, 72782, 52378, 62702, 64408, 94053, 27042, 5867, 20500, 25170, 62702, 81011, 8878, 34908], [40405, 93070, 55387, 40405, 40405, 40405, 40405, 40405, 40405, 40405, 64631], [98230, 16079, 52556, 55644, 21951, 98230, 55644, 83680, 6981, 94247, 94071, 25197, 12881, 39728, 40383, 52556, 6981, 26955, 95339, 47873, 32207, 21401], [65493, 59396, 48903, 98230, 36523, 2284, 59285, 44468, 87580, 5385, 13923], [40405, 40405, 97010, 49628, 12656, 78247, 96559, 2608], [47644, 21480, 10646, 79264, 9470, 78215, 19307, 38165, 68226, 5470, 94179, 97251, 5470,

In [121]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [122]:
## pre padding
sent_length=100
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[    0     0     0 ...  7220 80131 53232]
 [    0     0     0 ... 14652 57956 82532]
 [    0     0     0 ... 37556 80621 67617]
 ...
 [    0     0     0 ... 63131 21951 55560]
 [    0     0     0 ... 87905 15079  8261]
 [    0     0     0 ... 83680 31650 86589]]


In [126]:
df.label.unique()

array([0, 1], dtype=int64)

In [127]:
x_final = np.array(embedded_docs)
y_final = np.array(df['label'])
x_train,x_test,y_train,y_test = train_test_split(x_final,y_final,test_size=0.33, random_state=42)

In [128]:
sent_length=100
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [129]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10)

Epoch 1/10
670/670 [==============================] - 49s 72ms/step - loss: 0.1907 - accuracy: 0.9393 - val_loss: 0.1629 - val_accuracy: 0.9440
Epoch 2/10
670/670 [==============================] - 49s 74ms/step - loss: 0.0741 - accuracy: 0.9738 - val_loss: 0.1374 - val_accuracy: 0.9525
Epoch 3/10
670/670 [==============================] - 51s 76ms/step - loss: 0.0326 - accuracy: 0.9893 - val_loss: 0.1568 - val_accuracy: 0.9573
Epoch 4/10
670/670 [==============================] - 50s 75ms/step - loss: 0.0160 - accuracy: 0.9948 - val_loss: 0.1868 - val_accuracy: 0.9539
Epoch 5/10
670/670 [==============================] - 51s 76ms/step - loss: 0.0073 - accuracy: 0.9981 - val_loss: 0.2335 - val_accuracy: 0.9475
Epoch 6/10
670/670 [==============================] - 51s 76ms/step - loss: 0.0102 - accuracy: 0.9965 - val_loss: 0.1973 - val_accuracy: 0.9471
Epoch 7/10
670/670 [==============================] - 50s 75ms/step - loss: 0.0060 - accuracy: 0.9979 - val_loss: 0.2535 - val_accuracy:

In [130]:
from sklearn.metrics import classification_report
pred = model.predict(x_test)

330/330 [==============================] - 5s 13ms/step


In [131]:
pred

array([[6.891743e-07],
       [3.582063e-05],
       [5.333864e-07],
       ...,
       [8.027128e-01],
       [6.266883e-04],
       [3.805573e-06]], dtype=float32)

In [132]:
y_pred = np.where(pred>0.5,1,0)
y_pred

array([[0],
       [0],
       [0],
       ...,
       [1],
       [0],
       [0]])

In [133]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9466249525976489

In [141]:
text = "its unbelievable that in the 21st century we'd need something like this. again. #neverump  #xenophobia "
onehot_repr=one_hot(text,voc_size)
embedded_doc=pad_sequences([onehot_repr],padding='pre',maxlen=sent_length)
if model.predict(embedded_doc) >0.5:
    print('negative tweet')
else:
    print('positive tweet')

1/1 [==============================] - 0s 15ms/step
positive tweet


In [142]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[9509,  297],
       [ 266,  476]], dtype=int64)

In [143]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9466249525976489

In [144]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      9806
           1       0.62      0.64      0.63       742

    accuracy                           0.95     10548
   macro avg       0.79      0.81      0.80     10548
weighted avg       0.95      0.95      0.95     10548

